In [21]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import random
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [22]:
project_path = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\sem4\ADV topics in DS LLM\Project\Text-to-image-for-compression'

# # 10 images 0 to 9 - 1 each
# img_path = project_path + r'\\Data\Data-10-images'
# model_name = 'nn-10-0to9.h5'
# grayscale_path = img_path + '-gray'

# # 10 images of 1
# img_path = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\sem4\ADV topics in DS LLM\Project\Text-to-image-for-compression\Data\Data-10-images-1'
# model_name = 'nn-10-1.h5'
# grayscale_path = img_path + '-gray'

# All images of 1 - 10772 images
img_path = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\sem4\ADV topics in DS LLM\Project\Text-to-image-for-compression\Data\dataset\1\1'
model_name = 'nn-all-1.h5'
grayscale_path = img_path + '-gray'

# # All images 0 to 9 - 118503 images
# img_path = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\sem4\ADV topics in DS LLM\Project\Text-to-image-for-compression\Data\Data-all-images'
# model_name = 'nn-all.h5'
# grayscale_path = img_path + '-gray'


model_save_path = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\sem4\ADV topics in DS LLM\Project\Text-to-image-for-compression'

# Create the folder if it doesn't exist
if not os.path.exists(grayscale_path):
    os.makedirs(grayscale_path)

image_extension = '.png'

data = []

# Loop through all directories and subdirectories
for root, dirs, files in os.walk(img_path):
    for file in files:
        if file.lower().endswith(image_extension):
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                
                # If the image is in RGBA, convert transparent pixels to white
                if img.mode == 'RGBA':
                    white_bg = Image.new('RGB', img.size, (255, 255, 255))
                    white_bg.paste(img, mask=img.split()[3])
                    # Convert the image to grayscale (L mode)
                    img = white_bg.convert('L')
                else:
                    # If the image is already grayscale or RGB, just convert it to grayscale
                    img = img.convert('L')
                
                # Convert image to numpy array and save to DataFrame
                img_data = np.array(img)
                label = file_path.split('\\')[-1]
                data.append({"label": label, "image_data": img_data})

                # Save the converted grayscale image
                save_path = os.path.join(grayscale_path, file)
                img.save(save_path)

            except Exception as e:
                print(f"Error opening {file_path}: {e}")

# Create a DataFrame
df = pd.DataFrame(data)
print(df.head())

      label                                         image_data
0     0.png  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...
1     1.png  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...
2    10.png  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...
3   100.png  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...
4  1000.png  [[255, 255, 255, 255, 255, 255, 255, 255, 255,...


In [23]:
# Extract labels from the dataframe
labels = df['label'].values

# Convert the labels to one-hot encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)

# onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# The onehot_encoded array will be used as input to the model

In [24]:
# Assuming all images are grayscale and of size
X = df['image_data'].values

# Normalize pixel values
X = np.array([x.flatten() / 255.0 for x in X])  # Flatten the images and normalize pixel values

In [25]:
# Define the model
def create_model(input_dim, output_dim):
    model = models.Sequential()
    model.add(layers.Dense(32, activation='relu', input_dim=input_dim))  # First dense layer
#     model.add(layers.Dense(512, activation='relu'))  # Hidden dense layer
#     model.add(layers.Dense(1024, activation='relu'))  # Hidden dense layer
    model.add(layers.Dense(32, activation='relu'))  # Hidden dense layer
    model.add(layers.Dense(output_dim, activation='sigmoid'))  # Output layer (flattened image)
    return model

# Assume one-hot encoded labels have shape (num_samples, num_labels)
input_dim = onehot_encoded.shape[1]
output_dim = 28 * 28

model = create_model(input_dim, output_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                344768    
                                                                 
 dense_7 (Dense)             (None, 32)                1056      
                                                                 
 dense_8 (Dense)             (None, 784)               25872     
                                                                 
Total params: 371,696
Trainable params: 371,696
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Train the model
model.fit(onehot_encoded, X, epochs=1200, batch_size=64)

Epoch 1/1200
1852/1852 [==============================] - 65s 35ms/step - loss: 0.0552 - accuracy: 0.0000e+00
Epoch 2/1200
1852/1852 [==============================] - 64s 34ms/step - loss: 0.0465 - accuracy: 0.0000e+00
Epoch 3/1200
1852/1852 [==============================] - 64s 34ms/step - loss: 0.0458 - accuracy: 0.0000e+00
Epoch 4/1200
1852/1852 [==============================] - 64s 34ms/step - loss: 0.0443 - accuracy: 8.4386e-06
Epoch 5/1200
1852/1852 [==============================] - 66s 35ms/step - loss: 0.0421 - accuracy: 0.0000e+00
Epoch 6/1200
1852/1852 [==============================] - 65s 35ms/step - loss: 0.0388 - accuracy: 0.0000e+00
Epoch 7/1200
1852/1852 [==============================] - 65s 35ms/step - loss: 0.0362 - accuracy: 0.0000e+00
Epoch 8/1200
1852/1852 [==============================] - 65s 35ms/step - loss: 0.0333 - accuracy: 0.0000e+00
Epoch 9/1200
1852/1852 [==============================] - 64s 35ms/step - loss: 0.0307 - accuracy: 0.0000e+00
Epoch 10/1

1852/1852 [==============================] - 69s 37ms/step - loss: 0.0128 - accuracy: 0.9515
Epoch 78/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0127 - accuracy: 0.9537
Epoch 79/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0127 - accuracy: 0.9547
Epoch 80/1200
1852/1852 [==============================] - 70s 38ms/step - loss: 0.0127 - accuracy: 0.9568
Epoch 81/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0126 - accuracy: 0.9585
Epoch 82/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0126 - accuracy: 0.9605
Epoch 83/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0126 - accuracy: 0.9617
Epoch 84/1200
1852/1852 [==============================] - 70s 37ms/step - loss: 0.0126 - accuracy: 0.9636
Epoch 85/1200
1852/1852 [==============================] - 70s 38ms/step - loss: 0.0125 - accuracy: 0.9646
Epoch 86/1200
1852/1852 [==========================

1852/1852 [==============================] - 69s 37ms/step - loss: 0.0114 - accuracy: 0.9950
Epoch 154/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0114 - accuracy: 0.9952
Epoch 155/1200
1852/1852 [==============================] - 70s 38ms/step - loss: 0.0114 - accuracy: 0.9952
Epoch 156/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0114 - accuracy: 0.9953
Epoch 157/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0113 - accuracy: 0.9956
Epoch 158/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0113 - accuracy: 0.9957
Epoch 159/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0113 - accuracy: 0.9958
Epoch 160/1200
1852/1852 [==============================] - 70s 38ms/step - loss: 0.0113 - accuracy: 0.9959
Epoch 161/1200
1852/1852 [==============================] - 69s 37ms/step - loss: 0.0113 - accuracy: 0.9958
Epoch 162/1200
1852/1852 [=================

KeyboardInterrupt: 

In [20]:
def check_if_float32(model):
    for layer in model.layers:
        weights = layer.get_weights()
        if weights:  # Some layers don't have weights
            for weight in weights:
                dtype = weight.dtype
                if dtype == 'float32':
                    print(f"Layer: {layer.name}, Data type is float32")
                else:
                    print(f"Layer: {layer.name}, Data type is {dtype}, not float32")

# Check if the model weights are float32
check_if_float32(model)

Layer: dense_3, Data type is float32
Layer: dense_3, Data type is float32
Layer: dense_4, Data type is float32
Layer: dense_4, Data type is float32
Layer: dense_5, Data type is float32
Layer: dense_5, Data type is float32


In [7]:
model.save_weights(model_save_path + '\\saved_weights\\' + model_name) # save weights only

# Save the architecture
model_json = model.to_json()
with open(model_save_path + '\\saved_architecture\\weights', "w") as json_file:
    json_file.write(model_json)

In [8]:
# model = tf.keras.models.load_model('\\nn-10-1.h5')

# Load the architecture from the file
from keras.models import model_from_json
with open(model_save_path + '\\saved_architecture\\weights', "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    
# Load the weights into the new model
model.load_weights(model_save_path + '\\saved_weights\\' + model_name)

In [9]:
# Compile the model (necessary if you're going to use it for training or evaluation)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                3792128   
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 784)               25872     
                                                                 
Total params: 3,819,056
Trainable params: 3,819,056
Non-trainable params: 0
_________________________________________________________________


In [15]:
def pixel_accuracy(original, predicted):
    """
    Calculate pixel accuracy based on the mean absolute error between the original and predicted images.

    Parameters:
    original (np.array): The original image array (shape: height x width).
    predicted (np.array): The predicted image array (shape: height x width).

    Returns:
    float: The pixel accuracy as a percentage.
    """
    original = np.array(original)
    predicted = np.array(predicted)
    
    # Check if the shapes match
    if original.shape != predicted.shape:
        raise ValueError("Original and predicted images must have the same shape.")
    
    # Calculate the mean absolute error
    mae = np.mean(np.abs(original - predicted))

    # Convert MAE to a percentage accuracy (assuming pixel values are in [0, 255])
    max_mae = 255
    accuracy = 100 * (1 - (mae / max_mae))  # Normalize accuracy to be between 0 and 100

    return accuracy  # Return as percentage

# Function to get the original image data from the DataFrame
def get_original_image_from_df(label, df):
    # Locate the row in the DataFrame that matches the label
    row = df[df['label'] == label]
    if not row.empty:
        return row['image_data'].values[0]  # Get the image data
    else:
        print("Label not found in DataFrame.")
        return None

# List to hold pixel accuracy values for all predictions
all_accuracies = []

# Make a prediction
# labels_to_predict = df['label'].values
labels_to_predict = random.sample(list(df['label'].values), 10)

for label in labels_to_predict:
    encoded_label = onehot_encoder.transform([[label_encoder.transform([label])[0]]])
    predicted_image = model.predict(encoded_label)

    # Reshape the predicted image and unnormalize it (convert back to 0-255 range)
    predicted_image = predicted_image.reshape((28, 28))
    predicted_image = predicted_image * 255.0  # Unnormalize the predicted image

    # Get the original image from the DataFrame
    original_image = get_original_image_from_df(label, df)

    # Check if original image is not None
    if original_image is not None:
        # Calculate pixel accuracy
        accuracy = pixel_accuracy(original_image, predicted_image)
        all_accuracies.append(accuracy)
        print(f"Pixel Accuracy for label '{label}': {accuracy:.2f}%")

        # Set up the plot
        plt.figure(figsize=(10, 5))

        # Plot original image
        plt.subplot(1, 2, 1)
        plt.imshow(original_image, cmap='gray')
        plt.title('Original Image')
        plt.axis('off')  # Hide axis

        # Plot predicted image
        plt.subplot(1, 2, 2)
        plt.imshow(predicted_image, cmap='gray')
        plt.title('Predicted Image')
        plt.axis('off')  # Hide axis

        plt.show()

# Calculate and print the average pixel accuracy
average_accuracy = np.mean(all_accuracies)
print(f"\nAverage Pixel Accuracy over all above images: {average_accuracy:.2f}%")

ValueError: in user code:

    File "C:\Users\Mahender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Mahender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Mahender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Mahender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\Mahender\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [12]:
# List to hold pixel accuracy values for all predictions
all_accuracies = []

# Encode all labels in the DataFrame
encoded_labels = onehot_encoder.transform([[label_encoder.transform([label])[0]] for label in df['label'].values])

# Make predictions for all encoded labels at once
predicted_images = model.predict(encoded_labels)

# Loop over all predicted images and corresponding original images
for i, label in enumerate(df['label'].values):
    predicted_image = predicted_images[i].reshape((28, 28))  # Reshape the predicted image
    predicted_image = predicted_image * 255.0  # Unnormalize the predicted image

    # Get the original image from the DataFrame
    original_image = get_original_image_from_df(label, df)

    # Check if original image is not None
    if original_image is not None:
        # Calculate pixel accuracy
        accuracy = pixel_accuracy(original_image, predicted_image)
        all_accuracies.append(accuracy)

# Calculate and print the average pixel accuracy
if all_accuracies:
    average_accuracy = np.mean(all_accuracies)
    print(f"\nAverage Pixel Accuracy over all images: {average_accuracy:.2f}%")
else:
    print("No valid images found for accuracy calculation.")

3704/3704 [==============================] - 3s 736us/step

Average Pixel Accuracy over all images: 97.45%
